In [1]:
import pandas as pd
import numpy as np
import json


import os
import subprocess
import threading

import requests

import time
import datetime

import ta

import pytz
from arctic import Arctic, TICK_STORE
from arctic.date import DateRange

store = Arctic('localhost')

if store.library_exists('daddy') == False:
    store.initialize_library('daddy', lib_type=TICK_STORE)

library = store['daddy']
library._chunk_size = 500000

def get_data(url, index, proxy):    
    global results
    global threads
        
    if proxy == None:
        res = requests.get(url, timeout=2)
    else:
        proxies = {
          "http": "http://" + proxy,
          "https": "https://" + proxy,
        }
        res = requests.get(url, proxies=proxies, timeout=2)
        
    results[index] = pd.DataFrame(json.loads(res.text))

def get_df(start_time, proxy=None, total_range=30):
    global threads
    global results
    
    start_time = pd.to_datetime(start_time).tz_localize(None)
    
    if start_time.date() == datetime.datetime.utcnow().date():
        urls = ["https://www.bitmex.com/api/v1/trade?symbol=XBTUSD&count={}&start={}&reverse=false&startTime={}".format(1000, i * 1000, start_time) for i in range(total_range)]
    else:
        urls = ["https://www.bitmex.com/api/v1/trade?symbol=XBTUSD&count={}&start={}&reverse=false&startTime={}&endTime={}".format(1000, i * 1000, start_time, pd.to_datetime(start_time.date() + pd.Timedelta(days=1))) for i in range(total_range)]
    
    threads = [None] * len(urls)
    results = [None] * len(urls)
    
    for i in range(len(threads)):
        threads[i] = threading.Thread(target=get_data, args=(urls[i], i, proxy))
        threads[i].start()
    
    for i in range(len(threads)):
        threads[i].join()

    df = pd.DataFrame()

    for curr_df in results:
        df = df.append(curr_df, ignore_index=True)
                    
    return df

def manual_scrape(scrape_from, sleep=True):
    print("Manual scrape for {}".format(scrape_from))
    proxy_df = pd.read_csv('proxies', sep=':', header=None)
    proxy_df.columns = ['proxy', 'port', 'username', 'password']

    proxy_df['proxy_string'] =  proxy_df['username'] + ":" + proxy_df['password'] + "@" + proxy_df['proxy'] + ":" + proxy_df['port'].astype(str)
    proxy_list = list(proxy_df['proxy_string'])
    at_once = len(proxy_list) + 1
    all_df = pd.DataFrame()
    completed = False
    
    while True:
        start_time = time.time()
        
        for i in range(at_once):
            if i == 0:
                curr_df = get_df(scrape_from)
            else:
                curr_df = get_df(scrape_from, proxy=proxy_list[i-1])
                
            all_df = all_df.append(curr_df, ignore_index=True)
            all_df = all_df.dropna(subset=['timestamp'], how='all')
            
            scrape_from = all_df.iloc[-1]['timestamp']
            print("Got {} data till {}".format(len(curr_df), scrape_from))
            
            if len(curr_df) < 1000:
                completed = True
                break
         
        total_time_taken = time.time() - start_time
        
        to_sleep = int(60 - total_time_taken) + 1
        
        if completed == True:
            break

        if to_sleep > 0:
            if sleep == True:
                print("Sleeping {} seconds".format(to_sleep))
                time.sleep(to_sleep)
        else:
            print("No need to sleep")
            
    
    all_df['timestamp'] = pd.to_datetime(all_df['timestamp'])
    all_df['timestamp'] = all_df['timestamp'].dt.tz_localize(None)
    all_df = all_df.sort_values('timestamp').reset_index(drop=True)
            
    return all_df

def aws_scrape(name):
    print("AWS Scrape for {}".format(name))
    url = "https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/trade/{}".format(name)
    r = requests.get(url)
    
    with open('temp', 'wb') as f:
        f.write(r.content)
        
    df = pd.read_csv('temp', compression='gzip')
    os.remove('temp')
    aws_df = df[df['symbol'] == 'XBTUSD']
    aws_df['timestamp'] = pd.to_datetime(aws_df['timestamp'], format="%Y-%m-%dD%H:%M:%S.%f")
    aws_df = aws_df.sort_values('timestamp').reset_index(drop=True)
    return aws_df

def get_bitmex_data(start, end, sleep=True):
    all_df = []

    for scrape_date in pd.date_range(start, end):
        if scrape_date.date() == datetime.datetime.utcnow().date() - pd.Timedelta(days=1):
            curr_time = datetime.datetime.utcnow()
            if curr_time.time() > datetime.time(5,41):
                df = aws_scrape(scrape_date.strftime("%Y%m%d.csv.gz"))
            else:
                df = manual_scrape(scrape_date, sleep=sleep)
        elif scrape_date.date() == datetime.datetime.utcnow().date():
            df = manual_scrape(scrape_date,  sleep=sleep)
        else:
            df = aws_scrape(scrape_date.strftime("%Y%m%d.csv.gz"))


        all_df.append(df)
    
    return pd.concat(all_df, axis=0)

/home/warproxxx/.local/lib/python3.8/site-packages/arctic/_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/home/warproxxx/.local/lib/python3.8/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
def update_trades():
    end = pd.to_datetime(datetime.datetime.utcnow()).date()
    original_start = end - pd.Timedelta(days=20)
    
    try:
        start = pd.to_datetime(library.max_date('trades').astimezone(pytz.UTC)).tz_localize(None)
        
        if start.hour == 23 and start.minute >= 58:
            start = pd.to_datetime(start.date() + pd.Timedelta(days=1))
    except:
        start = original_start

    while True:
        try:
            end = pd.to_datetime(datetime.datetime.utcnow())

            print("{} to {}".format(start, end))
            df = get_bitmex_data(start, end)
            df = df[['timestamp', 'symbol', 'side', 'size', 'price', 'homeNotional', 'foreignNotional']]
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df = df.set_index('timestamp')
            df = df.tz_localize(tz='UTC')
            library.write('trades', df)               
            break
        except Exception as e:
            print("Exception: {}. Retrying in 5 secs".format(str(e)))
            time.sleep(5)
    


In [3]:
update_trades()

2021-01-19 08:44:00.628000 to 2021-01-19 09:15:38.286513
Manual scrape for 2021-01-19 08:44:00.628000
Got 8738 data till 2021-01-19T09:15:35.032Z


NB treating all values as 'exists' - no longer sparse


Got 5 data till 2021-01-19T09:15:41.052Z


In [4]:
def get_significant_traders(df):
    df = df[['timestamp', 'side', 'homeNotional', 'foreignNotional']]
    df = df.groupby(['timestamp', 'side']).sum() 
    df = df.reset_index()
    df = df[df['foreignNotional'] > 500]
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['price'] = df['foreignNotional']/df['homeNotional']
    df = df.sort_values('timestamp')
    df = df.drop_duplicates()
    return df

def get_features(curr_df):
    ser = {}
    curr_df = curr_df.sort_values('timestamp')
    
    if len(curr_df) > 0:
        ser['open'] = curr_df.iloc[0]['price']
        ser['high'] = curr_df['price'].max()
        ser['low'] = curr_df['price'].min()
        ser['close'] = curr_df.iloc[-1]['price']
        ser['volume'] = curr_df['foreignNotional'].sum()
    else:
        ser['open'] = np.nan
        ser['high'] = np.nan
        ser['low'] = np.nan
        ser['close'] = np.nan
        ser['volume'] = np.nan
        
    buy_orders = curr_df[curr_df['side'] == 'Buy']
    sell_orders = curr_df[curr_df['side'] == 'Sell']

    total_buy = buy_orders['homeNotional'].sum()
    total_sell = sell_orders['homeNotional'].sum()
    total = total_buy + total_sell

    ser['buy_percentage'] = total_buy/total
    ser['buy_volume'] = total_buy
    ser['all_volume'] = total
    
    readable_bins = []
    

    readable_bins = [0, 2, 10, np.inf]
        
    readable_labels = ['small', 'medium', 'large']
    curr_df['new_range'] = pd.cut(curr_df['homeNotional'], readable_bins, include_lowest=True, labels=readable_labels).astype(str)
    
        
    for curr_range in set(readable_labels):
        group = curr_df[curr_df['new_range'] == curr_range]
        ser["percentage_{}".format(curr_range)] = group['homeNotional'].sum()/total
        buy_orders = group[group['side'] == 'Buy']
        ser['buy_percentage_{}'.format(curr_range)] = (buy_orders['homeNotional'].sum())/group['homeNotional'].sum()

    
        
    return pd.Series(ser)

def get_features_from_sig(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    minute_only = df['timestamp'].dt.minute.astype(str)
    minute_only_two = minute_only.apply(lambda x: str(x)[1:]) #there is a mistake here.
    df = df[~((minute_only == '9') | (minute_only_two == '9') | (minute_only == '8')  | (minute_only_two == '8'))]

    features = df.groupby(pd.Grouper(key='timestamp', freq="10Min", label='left')).apply(get_features)
    features = features.reset_index()

    features['timestamp'] = pd.to_datetime(features['timestamp'])
    features = features.drop_duplicates(subset=['timestamp'])
    features = features.sort_values('timestamp')
    features = features.dropna()
    return features

In [5]:
def get_intervaled_date(startTime):
    time_df = pd.DataFrame(pd.Series({'Time': startTime})).T
    return time_df.groupby(pd.Grouper(key='Time', freq="10Min", label='left')).sum().index[0]

In [6]:
#there is a timezone issue

In [7]:
last_date = pd.to_datetime(library.max_date('trades').astimezone(pytz.UTC)).tz_localize(None)

In [8]:
last_date

Timestamp('2021-01-19 09:15:41.052000')

In [9]:

minute = str(last_date.time().minute)

if len(minute) == 1:
    minute_only = int(minute)
else:
    minute_only = int(minute[1:])
    
if (minute_only <= 8):
    have_till_calc = last_date - pd.Timedelta(minutes=10)
else:
    have_till_calc = last_date

In [10]:
have_till = get_intervaled_date(have_till_calc)

In [11]:
have_till

Timestamp('2021-01-19 09:00:00', freq='10T')

In [12]:
min_date = pd.to_datetime(library.min_date('trades').astimezone(pytz.UTC)).tz_localize(None)
startTime = get_intervaled_date(min_date)

if os.path.isfile('data/features.csv'):
    startTime = pd.to_datetime(pd.read_csv('data/features.csv').iloc[-1]['timestamp']) + pd.Timedelta(minutes=10)



In [13]:
startTime

Timestamp('2021-01-19 08:40:00')

In [14]:
have_till = have_till.tz_localize(tz='UTC')

In [15]:
startTime = startTime.tz_localize(tz='UTC')

In [16]:
df = library.read('trades', date_range = DateRange(start=startTime, end=have_till + pd.Timedelta(minutes=10)))
df = df.tz_convert('UTC').tz_localize(None)
df = df.reset_index()
df = df.rename(columns={'index': 'timestamp'})

In [17]:
#calculate and save features
df = get_significant_traders(df)
features = get_features_from_sig(df)

features['change'] = ((features['close'] - features['open'])/features['open']) * 100
features = features[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'change', 'percentage_large', 'buy_percentage_large']]


In [18]:
features

,timestamp,open,high,low,close,volume,change,percentage_large,buy_percentage_large
0,2021-01-19 08:40:00,37425.149701,37467.216186,37313.432836,37439.161363,15600287.0,0.037439,0.328568,0.536217
1,2021-01-19 08:50:00,37397.157816,37397.157816,37105.751391,37299.515106,27698680.0,-0.261097,0.171958,0.104960
2,2021-01-19 09:00:00,37299.515106,37467.216186,37299.515106,37453.183521,9946733.0,0.411985,0.122384,0.588066


In [19]:
if os.path.isfile('data/features.csv'):
    old_features = pd.read_csv('data/features.csv')
    old_features['timestamp'] = pd.to_datetime(old_features['timestamp'])
    features = pd.concat([old_features, features])
    
features['macd'] = ta.trend.macd_signal(features['close'])
features['rsi'] = ta.momentum.rsi(features['close'])

In [20]:
features

,timestamp,open,high,low,close,volume,change,percentage_large,buy_percentage_large,macd,rsi
0,2020-12-30 00:00:00,27382.256298,27639.584700,27382.252259,27442.238102,45986153.0,0.219054,0.356011,0.532777,NaN,100.000000
1,2020-12-30 00:10:00,27517.886626,27739.255657,27510.316369,27739.249437,31607399.0,0.804432,0.375702,0.694009,NaN,100.000000
2,2020-12-30 00:20:00,27716.186253,27855.153203,27685.486670,27723.870508,36130446.0,0.027725,0.384736,0.593010,NaN,94.718324
3,2020-12-30 00:30:00,27770.063833,27831.895615,27670.167911,27685.492802,25868575.0,-0.304540,0.361751,0.114267,NaN,82.944989
4,2020-12-30 00:40:00,27708.506511,27754.649963,27631.941762,27670.171555,11114320.0,-0.138351,0.158667,0.354679,NaN,78.737280
...,...,...,...,...,...,...,...,...,...,...,...
2741,2021-01-19 08:20:00,37383.177570,37593.984962,37299.515106,37523.452158,61567553.0,0.375235,0.307893,0.564661,62.932523,68.903217
2742,2021-01-19 08:30:00,37523.452158,37650.602410,37299.515106,37327.360956,33262106.0,-0.522583,0.408413,0.485571,84.224596,62.260597
0,2021-01-19 08:40:00,37425.149701,37467.216186,37313.432836,37439.161363,15600287.0,0.037439,0.328568,0.536217,105.099403,64.369661
1,2021-01-19 08:50:00,37397.157816,37397.157816,37105.751391,37299.515106,27698680.0,-0.261097,0.171958,0.104960,122.150868,59.869086


In [21]:

if os.path.isfile('data/features.csv'):
    features.to_csv('data/features.csv', header=None, mode='a', index=None)
else:
    features.to_csv('data/features.csv', index=None)

In [63]:
features = pd.read_csv('data/features.csv')
features['timestamp'] = pd.to_datetime(features['timestamp'])
dupe = features.iloc[-1]
dupe['timestamp'] = dupe['timestamp'] + pd.Timedelta(minutes=10)
features = features.append(dupe, ignore_index=True)

<ipython-input-63-beb0378365d2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['timestamp'] = dupe['timestamp'] + pd.Timedelta(minutes=10)


In [ ]:
#now backtest and get current position passing parameters as value

In [9]:
#run update_trade every 7th minute and at start

In [11]:
#after done, run update trade again and backtest verify